## Imports

In [33]:
import pandas as pd
import pycountry

In [34]:
world = pd.read_csv('data/World Energy Consumption.csv')

## Pre-Processing Data

### Remove columns with more than 50% of null values

In [35]:
def remove_null_sup_to_50(data):
    for col in data.columns:
        if data[col].isnull().sum()>(len(data)/2):
            data.drop(columns=[col],inplace=True)
    return data

In [36]:
world = remove_null_sup_to_50(world)

### Get the list of countries and keep only these countries

In [37]:
countries = [country.name for country in pycountry.countries]

In [38]:
world = world.query("country.isin(@countries)")

### Keep only the important columns for our map

In [39]:
to_keep = ['country', 'year', 'coal_production', 'gas_production', 'oil_production']

In [40]:
world = world[to_keep]

In [41]:
world.head()

,country,year,coal_production,gas_production,oil_production
23,Afghanistan,1900,0.0,0.0,NaN
24,Afghanistan,1901,0.0,0.0,NaN
25,Afghanistan,1902,0.0,0.0,NaN
26,Afghanistan,1903,0.0,0.0,NaN
27,Afghanistan,1904,0.0,0.0,NaN


#### Most recent non-null value for Coal, for every country

In [42]:
# Using last, we keep the most recent non-null values 
coal_prod = pd.DataFrame(world.groupby('country')['coal_production'].last()).reset_index()

In [43]:
# Get the total production
coal_prod['Total_Prod'] = coal_prod['coal_production'].sum()

In [44]:
# Get a percentage of production for every country
coal_prod['Percentage'] = coal_prod['coal_production']/coal_prod['Total_Prod']

In [45]:
# Remove Total_Prod column
coal_prod = coal_prod.drop(columns=['Total_Prod'])

In [46]:
# Drop Nan
coal_prod = coal_prod.dropna()

# Rank the countries based on the percentage of production they represent
coal_prod['Rank'] = coal_prod['Percentage'].rank(ascending=False, method='dense').astype(int)

#### Most recent non-null value for Gas, for every country

In [47]:
# Using last, we keep the most recent non-null values 
gas_prod = pd.DataFrame(world.groupby('country')['gas_production'].last()).reset_index()

In [48]:
# Get the total production
gas_prod['Total_Prod'] = gas_prod['gas_production'].sum()

In [49]:
# Get a percentage of production for every country
gas_prod['Percentage'] = gas_prod['gas_production']/gas_prod['Total_Prod']

In [50]:
# Remove Total_Prod column
gas_prod = gas_prod.drop(columns=['Total_Prod'])

In [51]:
# Drop Nan
gas_prod = gas_prod.dropna()

# Rank the countries based on the percentage of production they represent
gas_prod['Rank'] = gas_prod['Percentage'].rank(ascending=False, method='dense').astype(int)

#### Most recent non-null value for Oil, for every country

In [52]:
# Using last, we keep the most recent non-null values 
oil_prod = pd.DataFrame(world.groupby('country')['oil_production'].last()).reset_index()

In [53]:
# Get the total production
oil_prod['Total_Prod'] = oil_prod['oil_production'].sum()

In [54]:
# Get a percentage of production for every country
oil_prod['Percentage'] = oil_prod['oil_production']/oil_prod['Total_Prod']

In [55]:
# Remove Total_Prod column
oil_prod = oil_prod.drop(columns=['Total_Prod'])

In [56]:
# Drop Nan
oil_prod = oil_prod.dropna()

# Rank the countries based on the percentage of production they represent
oil_prod['Rank'] = oil_prod['Percentage'].rank(ascending=False, method='dense').astype(int)

## Add Longitudes and Information

In [57]:
geo = pd.read_csv('data/country-coord.csv')

In [58]:
geo = geo[['Country','Latitude (average)', 'Longitude (average)']]

In [59]:
geo.columns = ['country', 'Latitude', 'Longitude']

### Merge with the dataframes to get information for every country

In [60]:
coal_prod = pd.merge(coal_prod, geo, on='country', how='inner')

gas_prod = pd.merge(gas_prod, geo, on='country', how='inner')

oil_prod = pd.merge(oil_prod, geo, on='country', how='inner')

## Multiply Percentages by 100 to get Percentages for the Map

In [95]:
coal_prod['Percentage'] = coal_prod['Percentage']*100

gas_prod['Percentage'] = gas_prod['Percentage']*100

oil_prod['Percentage'] = oil_prod['Percentage']*100

### Keep only 2 decimals

In [96]:
coal_prod['Percentage'] = round(coal_prod['Percentage'],2)

gas_prod['Percentage'] = round(gas_prod['Percentage'],2)

oil_prod['Percentage'] = round(oil_prod['Percentage'],2)

# Export

In [98]:
coal_prod.to_csv('frontend/pages/data/coal_prod.csv', index=False)

gas_prod.to_csv('frontend/pages/data/gas_prod.csv', index=False)

oil_prod.to_csv('frontend/pages/data/oil_prod.csv', index=False)